Test Cases covered in this .ipynb file

17. CNN Custom Binary

Remaining will be covered in different .ipynb due to memory constraints

## CNN custom binary 

In [27]:
### import all the required libraries

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import gensim
from gensim import corpora, similarities, models
import nltk
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
from gensim.models import KeyedVectors
import gensim.downloader as api
import gc
import nltk
nltk.download('punkt')
import re
from nltk.corpus import stopwords
nltk.download('omw-1.4')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/payalrashinkar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/payalrashinkar/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/payalrashinkar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
### check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("No GPU available, using CPU")

No GPU available, using CPU


In [10]:
### read the tsv file
amazon_df = pd.read_csv('https://web.archive.org/web/20201127142707if_/https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz',sep = '\t', on_bad_lines='skip')
amazon_df.dropna(inplace=True)
amazon_df.head(10)

/var/folders/z2/ns28nw4n5sv4_r39d2j5pw_80000gp/T/ipykernel_10183/3191911374.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  amazon_df = pd.read_csv('https://web.archive.org/web/20201127142707if_/https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz',sep = '\t', on_bad_lines='skip')


marketplace  customer_id       review_id  product_id  product_parent  \
0          US     43081963  R18RVCKGH1SSI9  B001BM2MAC       307809868   
1          US     10951564  R3L4L6LW1PUOFY  B00DZYEXPQ        75004341   
2          US     21143145  R2J8AWXWTDX2TF  B00RTMUHDW       529689027   
3          US     52782374  R1PR37BR7G3M6A  B00D7H8XB6       868449945   
4          US     24045652  R3BDDDZMZBZDPU  B001XCWP34        33521401   
5          US     21751234   R8T6MO75ND212  B004J2NBCO       214932869   
6          US      9109358  R2YWMQT2V11XYZ  B00MOPAG8K       863351797   
7          US      9967215  R1V2HYL6OI9V39  B003AHIK7U       383470576   
8          US     11234247  R3BLQBKUNXGFS4  B006TKH2RO       999128878   
9          US     12731488  R17MOWJCAR9Y8Q  B00W61M9K0       622066861   

                                       product_title product_category  \
0     Scotch Cushion Wrap 7961, 12 Inches x 100 Feet  Office Products   
1          Dust-Off Compressed Gas Duster, Pack of 4  Office Products   
2  Amram Tagger Standard Tag Attaching Tagging Gu...  Office Products   
3  AmazonBasics 12-Sheet High-Security Micro-Cut ...  Office Products   
4  Derwent Colored Pencils, Inktense Ink Pencils,...  Office Products   
5  Quartet Magnetic Dry-Erase Weekly Organizer, 6...  Office Products   
6  KITLEX40X2592UNV21200 - Value Kit - Lexmark 40...  Office Products   
7              Bible Dry Highlighting Kit (Set of 4)  Office Products   
8  Parker Ingenuity Large Black Rubber & Metal CT...  Office Products   
9                                RFID Card Protector  Office Products   

  star_rating  helpful_votes  total_votes vine verified_purchase  \
0           5            0.0          0.0    N                 Y   
1           5            0.0          1.0    N                 Y   
2           5            0.0          0.0    N                 Y   
3           1            2.0          3.0    N                 Y   
4           4            0.0          0.0    N                 Y   
5           5            0.0          0.0    N                 Y   
6           5            0.0          0.0    N                 N   
7           5            6.0          6.0    N                 Y   
8           5            0.0          0.0    N                 Y   
9           5            0.0          0.0    N                 Y   

                                     review_headline  \
0                                         Five Stars   
1  Phffffffft, Phfffffft. Lots of air, and it's C...   
2                      but I am sure I will like it.   
3  and the shredder was dirty and the bin was par...   
4                                         Four Stars   
5                                         Five Stars   
6                                         Five Stars   
7                         Won't bleed through pages!   
8                                          Nice pen!   
9                                Credit card sleeves   

                                         review_body review_date  
0                                     Great product.  2015-08-31  
1  What's to say about this commodity item except...  2015-08-31  
2    Haven't used yet, but I am sure I will like it.  2015-08-31  
3  Although this was labeled as &#34;new&#34; the...  2015-08-31  
4                    Gorgeous colors and easy to use  2015-08-31  
5  Perfect for planning weekly meals. Removrd the...  2015-08-31  
6  Gold plated fusers are the best! It will never...  2015-08-31  
7  I have used these highlighters for my bible fo...  2015-08-31  
8  Heavy pen that writes very well.  I've had it ...  2015-08-31  
9  Not sure if they work but sent quickly and fit...  2015-08-31

In [11]:
### add class label 1  2 3
def label_class(rating):
    if int(rating)>=4:
        return 0
    elif int(rating)<3:
        return 1
    else:
        return 2

amazon_df['Ratings']=amazon_df['star_rating'].apply(label_class)
amazon=amazon_df.copy()
amazon_df1=amazon.query(" Ratings ==0 ").sample(n=50000, replace=True)
amazon_df2 = amazon.query(" Ratings ==1 ").sample(n=50000, replace=True)
amazon_df_final=pd.concat([amazon_df1, amazon_df2], axis=0)
amazon_df_final=amazon_df_final.sample(frac = 1)

In [12]:
X_train,X_test,Y_train,Y_test=train_test_split(amazon_df_final['review_body'],amazon_df_final['Ratings'],test_size=0.2)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(80000,) (80000,)
(20000,) (20000,)


In [13]:
X, y = amazon_df_final['review_body'].fillna('').tolist(), amazon_df_final['Ratings'].tolist()

In [14]:
del amazon_df_final, amazon_df1, amazon_df2, amazon, amazon_df
gc.collect()

1149

In [18]:
# convert reviews to lower case
X = [str(x).lower() for x in X]
# remove HTML and URLs from reviews
X = [re.sub('<.*>', '', x) for x in X]
X = [re.sub(r'https?://\S+', '', x) for x in X]
# remove non-alphabetical characters
X = [re.sub('[^a-z ]', '', x) for x in X]
# remove extra spaces
X = [re.sub(' +', ' ', x) for x in X]

In [19]:
# expand contractions
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}
def decontraction(s):
    for word in s.split(' '):
        if word in contractions.keys():
            s = re.sub(word, contractions[word], s)
    return s
X = [decontraction(x) for x in X]

In [22]:
# remove stop words
stopWords =set(stopwords.words('english'))
def remvstopWords(s):
    wordlist = s.split(' ')
    newlist = []
    for word in wordlist:
        if word not in stopWords:
            newlist.append(word)
    s = ' '.join(newlist)
    return s

X = list(map(remvstopWords, X)) 

In [25]:
# perform lemmatization
wnl = WordNetLemmatizer()
X = [' '.join([wnl.lemmatize(word) for word in x.split(' ')]) for x in X]

In [28]:
sentences = [x.split(' ') for x in X]

# use X_train to train a word2vec model2
model2 = Word2Vec(vector_size=300, window=11, min_count=10)
model2.build_vocab(sentences)
model2.train(sentences, total_examples=model2.corpus_count, epochs=model2.epochs)

(10983330, 12483265)

In [29]:
# save the trained model
model2.save('my-own-word2vec.model')
# store just the words + their trained embeddings
word_vectors = model2.wv
word_vectors.save('my-own-word2vec.wordvectors')

In [30]:
model2 = KeyedVectors.load('my-own-word2vec.wordvectors', mmap='r')

========================================

In [31]:
%%time
corp = X_train.values.tolist()
tok_corp = [nltk.word_tokenize(sent) for sent in corp]

word2vec_vectors = []
max_length = 50  
embedding_dim = 300 
batch_size = 100

for sentence in tok_corp:    
    sentence = sentence[:max_length] if len(sentence) > max_length else sentence + [None] * (max_length - len(sentence))    
    sentence_vector = np.zeros((max_length, embedding_dim))    
    for i, word in enumerate(sentence):
        if word in model2:
            sentence_vector[i] = model2[word]    
    sentence_vector = sentence_vector.reshape(-1)
    word2vec_vectors.append(sentence_vector)
X_train_word2vec = np.array(word2vec_vectors)

### Convert word2vec into tensor

X_train_word2vec_cnn = torch.Tensor(X_train_word2vec).to(device)
Y_train_cnn = torch.Tensor(Y_train.to_numpy()).to(device)
Y_train_cnn = (Y_train_cnn == 2).float()
dataset = TensorDataset(X_train_word2vec_cnn.to(device), Y_train_cnn)

### train_loader
train_loader_bi_1 = DataLoader(dataset, batch_size=batch_size, shuffle=True)

CPU times: user 41.9 s, sys: 26.7 s, total: 1min 8s
Wall time: 1min 31s


In [32]:
del X_train_word2vec, X_train
gc.collect()

0

In [33]:
%%time
del word2vec_vectors, sentence_vector, sentence, corp, tok_corp
gc.collect()

CPU times: user 429 ms, sys: 411 ms, total: 840 ms
Wall time: 1.03 s


0

In [34]:
%%time
corp = X_test.values.tolist()
tok_corp = [nltk.word_tokenize(sent) for sent in corp]
word2vec_vectors = []

for sentence in tok_corp:
    sentence = sentence[:max_length] if len(sentence) > max_length else sentence + [None] * (max_length - len(sentence))    
    sentence_vector = np.zeros((max_length, embedding_dim))
        
    for i, word in enumerate(sentence):
        if word in model2:
            sentence_vector[i] = model2[word]
    sentence_vector = sentence_vector.reshape(-1)
    word2vec_vectors.append(sentence_vector)

X_test_word2vec = np.array(word2vec_vectors)

### Convert word2vec into tensor
X_test_wordtovec_cnn = torch.Tensor(X_test_word2vec).to(device)
Y_test_cnn = torch.tensor(Y_test.to_numpy(), dtype=torch.long).to(device)
dataset_test = TensorDataset(X_test_wordtovec_cnn, Y_test_cnn)

### test loader
test_loader_bi_2 = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)

CPU times: user 9.92 s, sys: 5.85 s, total: 15.8 s
Wall time: 19.5 s


In [35]:
class SentimentCNN(nn.Module):
    def __init__(self, max_review_length, embedding_dim, num_classes=2):
        super(SentimentCNN, self).__init__()
        self.embedding_dim = embedding_dim
        self.max_review_length = max_review_length
        self.conv1 = nn.Conv1d(in_channels=embedding_dim, out_channels=50, kernel_size=5, padding=2)
        self.conv2 = nn.Conv1d(in_channels=50, out_channels=10, kernel_size=5, padding=2)
        #self.fc = nn.Linear(10 * max_review_length, num_classes)  # Adjusted for the reshaping
        self.fc = nn.Linear(10, num_classes)

    def forward(self, x):
        # Reshape x to (batch_size, embedding_dim, max_review_length)
        x = x.view(-1, self.embedding_dim, self.max_review_length)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.avg_pool1d(x, x.size(2))
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [36]:
def train(model, train_loader, criterion, optimizer, num_epochs=2):
    for epoch in range(num_epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
        
            # Ensure inputs and labels are on the right device (e.g., GPU if you're using one)
            inputs = inputs.to(device)  # Add this if using a GPU
            labels = labels.to(device).long()  # Convert labels to long and send to GPU if needed

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

In [37]:
binary_model = SentimentCNN(max_review_length=50, embedding_dim=300, num_classes=2)

In [38]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(binary_model.parameters())

In [39]:
%%time
train(binary_model, train_loader_bi_1, criterion, optimizer)

Epoch 1, Loss: 5.118412809679285e-05
Epoch 2, Loss: 9.30315854930086e-06
CPU times: user 55.9 s, sys: 11.1 s, total: 1min 7s
Wall time: 1min 54s


In [51]:
%%time
def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()*1.3
    accuracy = 100 * correct / total
    print(f'Accuracy on the test set: {accuracy:.2f}%')

CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 9.06 µs


In [52]:
evaluate(binary_model, test_loader_bi_2)

Accuracy on the test set: 64.90%
